In [2]:
import pandas as pd
from tqdm import tqdm
pd.set_option("display.max_columns",100)

### 1. Load Datasets

In [3]:
## Load Penalty Data
plays_df = pd.read_csv('data/game_plays.csv')
penalties_df = plays_df[plays_df.event == 'Penalty']

#Load Team Info
team_info = pd.read_csv('data/team_info.csv')
team_info['combined_name'] = team_info.shortName + ' ' + team_info.teamName
team_info.drop(['franchiseId','shortName','teamName','abbreviation','link'],
               axis = 1,inplace=True)

#Load Games Dataset
games_df = pd.read_csv('data/game.csv')
games_df.drop(['venue_link','venue_time_zone_id'], axis= 1, inplace= True)

#Load and Drop Unused Columns
team_info = pd.read_csv('data/team_info.csv')
team_info['combined_name'] = team_info.shortName + ' ' + team_info.teamName
team_info.drop(['franchiseId','shortName','teamName','abbreviation','link'],
               axis = 1,inplace=True)

### 2. Create Home/Away and Join

In [4]:
#Create home and away datasets for joining
away_info = team_info.copy()
away_info.columns = ['away_team_id','away_team_name']
home_info = team_info.copy()
home_info.columns = ['home_team_id','home_team_name']

#Merge Columns
games_df = games_df.merge(away_info)
games_df = games_df.merge(home_info)
penalties_df = penalties_df.merge(games_df,on='game_id')

## Get unique games
unique_games = penalties_df.game_id.unique()
print("Loaded {} Unique Games".format(len(unique_games)))

Loaded 7293 Unique Games


### Create For, Winner, and Leading@time Columns

In [5]:
penalties_df['for'] = 'mismatch'
penalties_df.loc[penalties_df.team_id_for == penalties_df.home_team_id,
                 'for']= 'home'
penalties_df.loc[penalties_df.team_id_for == penalties_df.away_team_id,
                 'for']= 'away'

penalties_df['winner'] = 'even'
penalties_df.loc[penalties_df['home_goals'] > penalties_df['away_goals'],
                 'winner'] = "home"
penalties_df.loc[penalties_df['away_goals'] > penalties_df['home_goals'],
                 'winner'] = "away"

penalties_df['leading_at_time'] = 'even'
penalties_df.loc[penalties_df['goals_home'] > penalties_df['goals_away'],
                 'leading_at_time'] = "home"
penalties_df.loc[penalties_df['goals_away'] > penalties_df['goals_home'],
                 'leading_at_time'] = "away"

### 3. Iterate Over Games and  Produce Final DF

In [6]:
all_last_pens = []
for gameid in tqdm(unique_games):
    game_pens = penalties_df[penalties_df.game_id == gameid].sort_values('play_num')
    game_plays =plays_df[plays_df.game_id == gameid].sort_values('play_num')
    game_pens.reset_index(inplace = True)
    last_pen_list = []
    for ix,row in game_pens.iterrows():
        if ix == 0:
            last_pen_list = ["First Penalty"]
        else:
            last_pen_list.append(last_pen)
        last_pen = row['for']
    all_last_pens = all_last_pens + last_pen_list
    
penalties_df['last_penalty_for'] = all_last_pens

100%|██████████| 7293/7293 [00:32<00:00, 222.35it/s]


In [25]:
penalties_df['counter'] = 1
penalties_df.groupby(['last_penalty_for','leading_at_time','for']).agg({'counter':'sum'})

counter
last_penalty_for leading_at_time for          
First Penalty    away            away      659
                                 home      588
                 even            away     2498
                                 home     2177
                 home            away      683
                                 home      688
away             away            away     3719
                                 home     4224
                 even            away     3333
                                 home     4569
                 home            away     4595
                                 home     6527
home             away            away     5134
                                 home     3021
                 even            away     4619
                                 home     2736
                 home            away     5518
                                 home     4008

In [16]:
# Two of the primary observations in this paper are that (1) teams that have taken more
# penalties in a match are less likely to have the next penalty called against them and (2)
# teams that are leading in a match are more likely to have the next penalty called against
# them

# next penalty more likely on the road team
# I next penalty more likely on the team with fewer penalties
# I next penalty more likely on the team having scored more goals
# I next penalty more likely on the weaker team
# I as matches progress, the penalty differential effect decreases